In [1]:
from datasets import load_dataset

c:\Users\HP\miniconda3\envs\final\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset('smilegate-ai/kor_unsmile')

Downloading: 100%|██████████| 1.58k/1.58k [00:00<00:00, 1.62MB/s]
Using custom data configuration smilegate-ai--kor_unsmile-1dba960877497f9f


Downloading: 100%|██████████| 1.16M/1.16M [00:00<00:00, 1.19MB/s]
Downloading: 100%|██████████| 290k/290k [00:00<00:00, 41.5MB/s]
100%|██████████| 2/2 [00:00<00:00, 2054.02it/s]


Dataset parquet downloaded and prepared to C:\Users\HP\.cache\huggingface\datasets\parquet\smilegate-ai--kor_unsmile-1dba960877497f9f\0.0.0\1638526fd0e8d960534e2155dc54fdff8dce73851f21f031d2fb9c2cf757c121. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 76.51it/s]


In [3]:
dataset["train"][0]

{'문장': '일안하는 시간은 쉬고싶어서 그런게 아닐까',
 '여성/가족': 0,
 '남성': 0,
 '성소수자': 0,
 '인종/국적': 0,
 '연령': 0,
 '지역': 0,
 '종교': 0,
 '기타 혐오': 0,
 '악플/욕설': 0,
 'clean': 1,
 '개인지칭': 0,
 'labels': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]}

In [4]:
unsmile_labels = ["여성/가족","남성","성소수자","인종/국적","연령","지역","종교","기타 혐오","악플/욕설","clean"]
# 개인지칭의 경우, 추가 정보이므로 분류 대상에서 제외했습니다.

In [30]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

  2%|▏         | 19/1175 [11:46<11:56:32, 37.19s/it]


In [31]:
from transformers import TextClassificationPipeline, BertForSequenceClassification, AutoTokenizer

model_name = 'smilegate-ai/kor_unsmile'

model = BertForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

pipe = TextClassificationPipeline(
    model=model,
    tokenizer=tokenizer,
    device=0,     # cpu: -1, gpu: gpu number
    return_all_scores=True,
    function_to_apply='sigmoid'
    )

loading configuration file config.json from cache at C:\Users\HP/.cache\huggingface\hub\models--smilegate-ai--kor_unsmile\snapshots\f597423eeff8e6da99cad85cbe3a81adf5225637\config.json
Model config BertConfig {
  "_name_or_path": "beomi/kcbert-base",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "\uc5ec\uc131/\uac00\uc871",
    "1": "\ub0a8\uc131",
    "2": "\uc131\uc18c\uc218\uc790",
    "3": "\uc778\uc885/\uad6d\uc801",
    "4": "\uc5f0\ub839",
    "5": "\uc9c0\uc5ed",
    "6": "\uc885\uad50",
    "7": "\uae30\ud0c0 \ud610\uc624",
    "8": "\uc545\ud50c/\uc695\uc124",
    "9": "clean"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "clean": 9,
    "\uae30\ud0c0 \ud610\uc624": 7,
    "\ub0a8\uc131": 1,
    "\uc131\uc18c\uc218\uc790": 2,
   

In [32]:
for result in pipe("이래서 여자는 게임을 하면 안된다")[0]:
    print(result)

{'label': '여성/가족', 'score': 0.8253052234649658}
{'label': '남성', 'score': 0.039725154638290405}
{'label': '성소수자', 'score': 0.01214433740824461}
{'label': '인종/국적', 'score': 0.023181872442364693}
{'label': '연령', 'score': 0.010315308347344398}
{'label': '지역', 'score': 0.018454890698194504}
{'label': '종교', 'score': 0.011270025745034218}
{'label': '기타 혐오', 'score': 0.0207340307533741}
{'label': '악플/욕설', 'score': 0.057331446558237076}
{'label': 'clean', 'score': 0.14010532200336456}


In [48]:
len(pipe("게임을 뭐 이따구로 만드냐 진짜 할맛 안난다")[0])

c:\Users\HP\miniconda3\envs\final\lib\site-packages\transformers\pipelines\base.py:1046: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


10

c:\Users\HP\miniconda3\envs\final\lib\site-packages\transformers\pipelines\base.py:1046: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


TypeError: '>' not supported between instances of 'dict' and 'dict'

In [33]:
def get_predicated_label(output_labels, min_score):
    labels = []
    for label in output_labels:
        if label['score'] > min_score:
            labels.append(1)
        else:
            labels.append(0)
    return labels

In [34]:
import tqdm
from transformers.pipelines.base import KeyDataset

predicated_labels = []

for out in tqdm.tqdm(pipe(KeyDataset(dataset['valid'], '문장'))):
    predicated_labels.append(get_predicated_label(out, 0.5))

Disabling tokenizer parallelism, we're using DataLoader multithreading already
100%|██████████| 3737/3737 [00:26<00:00, 140.66it/s]


In [35]:
from sklearn.metrics import classification_report

print(classification_report(dataset['valid']['labels'], predicated_labels))

              precision    recall  f1-score   support

           0       0.85      0.70      0.76       394
           1       0.87      0.83      0.85       334
           2       0.90      0.78      0.83       280
           3       0.87      0.79      0.82       426
           4       0.92      0.75      0.83       146
           5       0.87      0.88      0.88       260
           6       0.87      0.86      0.87       290
           7       0.92      0.18      0.30       134
           8       0.76      0.59      0.67       786
           9       0.74      0.79      0.77       935

   micro avg       0.82      0.73      0.77      3985
   macro avg       0.86      0.72      0.76      3985
weighted avg       0.82      0.73      0.77      3985
 samples avg       0.76      0.74      0.75      3985



c:\Users\HP\miniconda3\envs\final\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [5]:
from datasets import load_dataset
from transformers import BertForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer
import torch
import numpy as np

In [6]:
dataset = load_dataset('smilegate-ai/kor_unsmile')

Using custom data configuration smilegate-ai--kor_unsmile-1dba960877497f9f
Reusing dataset parquet (C:\Users\HP\.cache\huggingface\datasets\parquet\smilegate-ai--kor_unsmile-1dba960877497f9f\0.0.0\1638526fd0e8d960534e2155dc54fdff8dce73851f21f031d2fb9c2cf757c121)
100%|██████████| 2/2 [00:00<00:00, 683.67it/s]


In [7]:
model_name = 'beomi/kcbert-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading: 100%|██████████| 49.0/49.0 [00:00<?, ?B/s]
c:\Users\HP\miniconda3\envs\final\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Downloading: 100%|██████████| 619/619 [00:00<00:00, 636kB/s]
Downloading: 100%|████████

In [8]:
def preprocess_function(examples):
    tokenized_examples = tokenizer(str(examples["문장"]))
    tokenized_examples['labels'] = torch.tensor(examples["labels"], dtype=torch.float)
    # multi label classification 학습을 위해선 label이 float 형태로 변형되어야 합니다.
    # huggingface datasets 최신 버전에는 'map' 함수에 버그가 있어서 변형이 올바르게 되지 않습니다.
    
    return tokenized_examples

In [9]:
tokenized_dataset = dataset.map(preprocess_function)
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'labels', 'attention_mask', 'token_type_ids'])

Parameter 'function'=<function preprocess_function at 0x0000015F54EC4B88> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
100%|██████████| 3737/3737 [00:00<00:00, 5355.60ex/s]


In [10]:
tokenized_dataset['train'][0]

{'labels': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 'input_ids': tensor([    2,  2458, 15751, 24930, 24351, 29278, 17038, 11631,     3]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1])}

In [11]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [12]:
num_labels=len(unsmile_labels) # Label 갯수

model = BertForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=num_labels, 
    problem_type="multi_label_classification"
)
model.config.id2label = {i: label for i, label in zip(range(num_labels), unsmile_labels)}
model.config.label2id = {label: i for i, label in zip(range(num_labels), unsmile_labels)}

Downloading: 100%|██████████| 438M/438M [00:05<00:00, 86.9MB/s] 
Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model)

In [13]:
model.config.label2id

{'여성/가족': 0,
 '남성': 1,
 '성소수자': 2,
 '인종/국적': 3,
 '연령': 4,
 '지역': 5,
 '종교': 6,
 '기타 혐오': 7,
 '악플/욕설': 8,
 'clean': 9}

In [15]:
from sklearn.metrics import label_ranking_average_precision_score

In [16]:
def compute_metrics(x):
    return {
        'lrap': label_ranking_average_precision_score(x.label_ids, x.predictions),
    }

In [20]:
batch_size = 32 # 64 batch는 colab pro에서 테스트되었습니다.

In [21]:
args = TrainingArguments(
    output_dir="/content/drive/MyDrive/model/unsmile",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='lrap',
    greater_is_better=True,
)

trainer = Trainer(
    model=model, 
    args=args, 
    train_dataset=tokenized_dataset["train"], 
    eval_dataset=tokenized_dataset["valid"], 
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [23]:
from transformers import TextClassificationPipeline

pipe = TextClassificationPipeline(
    model = model,
    tokenizer = tokenizer,
    device=0,
    return_all_scores=True,
    function_to_apply='sigmoid'
    )

c:\Users\HP\miniconda3\envs\final\lib\site-packages\transformers\pipelines\text_classification.py:92: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  UserWarning,


In [24]:
for result in pipe("이래서 여자는 게임을 하면 안된다")[0]:
    print(result)

{'label': '여성/가족', 'score': 0.15060104429721832}
{'label': '남성', 'score': 0.18680186569690704}
{'label': '성소수자', 'score': 0.11889719218015671}
{'label': '인종/국적', 'score': 0.153749018907547}
{'label': '연령', 'score': 0.10417928546667099}
{'label': '지역', 'score': 0.11103798449039459}
{'label': '종교', 'score': 0.16230972111225128}
{'label': '기타 혐오', 'score': 0.10309185087680817}
{'label': '악플/욕설', 'score': 0.31387609243392944}
{'label': 'clean', 'score': 0.2767322063446045}


In [ ]:
def get_predicated_label(output_labels, min_score):
    labels = []
    for label in output_labels:
        if label['score'] > min_score:
            labels.append(1)
        else:
            labels.append(0)
    return labels

In [ ]:
from transformers import TextClassificationPipeline

pipe = TextClassificationPipeline(
    model = model,
    tokenizer = tokenizer,
    device=0,
    return_all_scores=True,
    function_to_apply='sigmoid'
    )

In [ ]:
for result in pipe("이래서 여자는 게임을 하면 안된다")[0]:
    print(result)